In [1]:
from datasets import load_dataset
from textbook_transformers import *

In [11]:
ds = load_dataset("gogo8232/fineweb-CC-MAIN-2024-10-insurance-700k-dedup", cache_dir = "/root/LLM_Playground/.huggingface_cache/datasets", split = "train")

In [3]:
import os

from groq import Groq

client_1 = Groq(
    api_key="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",
)

client_2 = Groq(
    api_key = "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"
)

client_3 = Groq(
    api_key= "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"
)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

def truncate(examples):
    if examples['token_count'] > 4096:
        tokenized = tokenizer.encode(examples['text'], max_length= 4096, truncation=True, add_special_tokens= False)
        examples['text'] = tokenizer.decode(tokenized)
    return examples

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
ds = ds.select(range(400, ds.num_rows))
with open('outputs.json') as f:
    outputs = json.load(f)

In [13]:
len(outputs)

400

In [14]:
from tqdm.auto import tqdm
import time
# outputs = []
for example in tqdm(ds, total = ds.num_rows):
    example = truncate(example)
    document = example['text']
    client = random.choice([client_1, client_2, client_3])
    suggestions = get_suggestions(document, client)
    client = random.choice([client_1, client_2, client_3])
    generator_kwargs = synthetic_textbook_function_calling(document, suggestions, client)
    transformation_prompt, generator_name = retrieve_prompt(document, **generator_kwargs)
    example.update({"chosen_transformation": generator_name})
    client = random.choice([client_1, client_2, client_3])
    model = random.choice(["llama3-70b-8192", "llama3-groq-70b-8192-tool-use-preview"])
    chat_completion = client.chat.completions.create(
        messages= transformation_prompt, 
        model=model,
        temperature = .7,
        top_p = .95,
    )
    transformed = chat_completion.choices[0].message.content
    # if random.random() > .9:
    #     try:
    #         client = random.choice([client_1, client_2])
    #         chat_completion = client.chat.completions.create(
    #             messages= transformation_prompt + [{"role": "system", "content": transformed}, {"role": "user", "content": "Make it at least 2x longer and comprehensive"}], 
    #             model="mixtral-8x7b-32768",
    #             max_tokens= 32768,
    #             temperature = .7,
    #             top_p = .95
    #         )
    #         example.update({"transformed_text": chat_completion.choices[0].message.content})
    #     except:
    #         example.update({"transformed_text": transformed})
    # else:
    example.update({"transformed_text": transformed})
    outputs.append(example)
    # if len(outputs) % 4 == 0:
        # time.sleep(3)
    if len(outputs) % 10 == 0:
        with open("outputs.json", "w") as f:
            json.dump(outputs, f)
        # time.sleep(5)

  0%|          | 0/745735 [00:00<?, ?it/s]